# orchestrator worker 
a central dynamically breakdown task, assgn them to worker llm and synthesizes their results.

or 
One central brain (orchestrator) breaks a big task into smaller tasks,
sends them to specialized workers,
then combines their outputs into one final answer.

In [2]:
#AI Research Assistant
# For Example User asks: Give me a short market analysis for an AI startup in healthcare.”


import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing_extensions import TypedDict
from langgraph.graph import START,StateGraph,END

load_dotenv()
os.environ["GROQ_API_kEY"]=os.getenv("GROQ_API_KEY")
llm=ChatGroq(model="llama-3.1-8b-instant")


#This state stores:
# original query , outputs from each worker ,final combined answer
class State(TypedDict):
    query:str
    market:str
    risks:str
    opportunities:str
    final_report:str 
    
def market_worker(state:State):
    msg=llm.invoke(f"Analyze the market for:",{state["query"]})
    return {"market":msg.content}

def risk_worker(state:State):
    msg=llm.invoke(f"Identify risk for:",{state["query"]})
    return {"risks":msg.content}

def opportunities_worker(state:State):
    msg=llm.invoke(f"Find the opportunities for:",{state["query"]})
    return {"opportunities":msg.content}


def orchestrator(state: State):
    # Orchestrator does NOT generate content
    # It just triggers workers
    return {}

def synthesizer(state: State):
    result = llm.invoke(
        f"""
        Create a short market report using:
        Market: {state['market']}
        Risks: {state['risks']}
        Opportunities: {state['opportunities']}
        """
    )
    return {"final_report": result.content}

In [ ]:
graph=StateGraph(State)

graph.add_node("market",market_worker)
graph.add_node("risks",risk_worker)
graph.add_node("opportunities",opportunities_worker)
graph.add_node("synthesizer",synthesizer)

# Start → all workers (parallel)
graph.add_edge(START, "market_worker")
graph.add_edge(START, "risk_worker")
graph.add_edge(START, "opportunity_worker")

# Workers → synthesizer
graph.add_edge("market_worker", "synthesizer")
graph.add_edge("risk_worker", "synthesizer")
graph.add_edge("opportunity_worker", "synthesizer")

# End
graph.add_edge("synthesizer", END)

